In [1]:
import json
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset

import json
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel

In [2]:
DATA_DIR = Path("data")
RECIPES_DATA = DATA_DIR / "recipe-ingredients-dataset.csv"
df = pd.read_csv(RECIPES_DATA)[:2500]

In [3]:
df["tags"] = df["tags"].apply(lambda x: json.loads(x.replace('\'', '"').replace('{', '[').replace('}', ']')))
df["search_terms"] = df["search_terms"].apply(lambda x: json.loads(x.replace('\'', '"').replace('{', '[').replace('}', ']')))
df["ingredients"] = df["ingredients"].apply(lambda x: json.loads(x.replace('\'', '"')))

In [4]:
df["tags_str"] = df["tags"].apply(lambda x: " ".join(x))
df["search_terms_str"] = df["search_terms"].apply(lambda x: " ".join(x))
df["ingredients_str"] = df["ingredients"].apply(lambda x: " ".join(x))

In [5]:
df["all_tags"] = df["description"] + " " + df["ingredients_str"] + " " + df["tags_str"] + " " + df["search_terms_str"] + " " + df["cuisine"]

In [6]:
df.drop(df[df['all_tags'].isnull()].index, inplace=True)
df = df.reset_index()

In [7]:
df

,index,Unnamed: 0,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,cuisine,tags_str,search_terms_str,ingredients_str,all_tags
0,0,0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","[water, grits, salt, cheddar cheese, garlic, o...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","[time-to-make, course, main-ingredient, prepar...","[diabetic, low-calorie, vegetarian, low-carb, ...",southern_us,time-to-make course main-ingredient preparatio...,diabetic low-calorie vegetarian low-carb side,water grits salt cheddar cheese garlic olive oil,"We love grits, this is another good way to ser..."
1,1,1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","[onion, red bell pepper, garlic cloves, large ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","[60-minutes-or-less, time-to-make, course, mai...","[dinner, shrimp]",cajun_creole,60-minutes-or-less time-to-make course main-in...,dinner shrimp,onion red bell pepper garlic cloves large shri...,"Simple, easy and very tasty for when you are i..."
2,3,3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"[zucchini, yellow squash, diced tomatoes, onio...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"[weeknight, time-to-make, course, main-ingredi...","[side, vegetarian, italian]",italian,weeknight time-to-make course main-ingredient ...,side vegetarian italian,zucchini yellow squash diced tomatoes onion ga...,This is a good recipe for weight watchers. It ...
3,4,4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"[beef stew meat, flour, salt, allspice, cinnam...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","[time-to-make, course, main-ingredient, prepar...",[dinner],russian,time-to-make course main-ingredient preparatio...,dinner,beef stew meat flour salt allspice cinnamon bl...,This is a fabulous stew that came from one of ...
4,5,5,232050,Hot Sweet Almond Brittle,This is one of our standard holiday gift recip...,"[slivered almonds, cider vinegar, sugar, sugar...","[""12 ounces slivered almonds"",""1/4 cup ...",1 (832 g),1,['Preheat oven to 375°F Place almonds in sing...,"[time-to-make, course, preparation, desserts, ...",[dessert],indian,time-to-make course preparation desserts candy...,dessert,slivered almonds cider vinegar sugar sugar sal...,This is one of our standard holiday gift recip...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,2495,2495,350629,Belgian White Beer Cookies With Orange Icing,I haven't tried this recipe yet but it sounds ...,"[beer, honey, unsalted butter, powdered sugar,...","["""",""2 (12 ounce) bottles belgian style whi...",1 (133 g),12,['Put the beer and honey to a medium saucepan ...,"[60-minutes-or-less, time-to-make, course, pre...","[cookie, dessert]",cajun_creole,60-minutes-or-less time-to-make course prepara...,cookie dessert,beer honey unsalted butter powdered sugar egg ...,I haven't tried this recipe yet but it sounds ...
2454,2496,2496,397738,Homestyle Lamb Shanks,"I'm a HUGE Triple D (Diner's, Drive-In's & Div...","[lamb shanks, extra virgin olive oil, beef bro...","[""4 lamb shanks"",""4 -6 tablespoons ...",1 (742 g),4,"['The night before, season the lamb shanks wit...","[course, main-ingredient, main-dish, lamb-shee...","[lamb, dinner]",greek,course main-ingredient main-dish lamb-sheep meat,lamb dinner,lamb shanks extra virgin olive oil beef broth ...,"I'm a HUGE Triple D (Diner's, Drive-In's & Div..."
2455,2497,2497,416351,Peppermint Patties,These Peppermint Patties are both Vegan and Gl...,"[coconut oil, agave nectar, peppermint oil, da...","[""1/2 cup coconut oil"","

In [12]:
with open('vectorizers/recipe_vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)
    
vectorized_data = tfidf.fit_transform(df['all_tags'].values)
vectorized_dataframe = pd.DataFrame(vectorized_data.toarray(), index=df['all_tags'].index.tolist())
vectorized_dataframe.shape

(2458, 5000)

In [13]:
with open('models/recipe_svd.pkl', 'rb') as f:
    svd = pickle.load(f)

reduced_data = svd.fit_transform(vectorized_dataframe)
reduced_data

array([[ 3.92802782e-01,  2.15764664e-02,  6.48547165e-02, ...,
         4.18198376e-03,  7.58921589e-04,  2.07543336e-04],
       [ 1.82352773e-01, -1.79778141e-01,  1.31466662e-01, ...,
        -8.07056931e-04,  4.79051664e-04, -1.43780763e-03],
       [ 1.70695804e-01, -1.37975042e-01,  8.00314656e-02, ...,
        -3.33299527e-03,  9.06598467e-04, -3.90840739e-03],
       ...,
       [ 1.03662590e-01, -8.11749752e-02, -1.01764389e-01, ...,
         5.81101014e-04, -2.83380372e-03,  4.99889316e-05],
       [ 4.23148782e-01,  1.19933993e-01,  8.91905702e-02, ...,
         2.75609964e-03, -5.00081694e-03, -1.40779357e-03],
       [ 1.96863382e-01, -1.12702043e-01,  5.28976102e-02, ...,
        -3.43964303e-03, -3.34987518e-03, -1.06503070e-03]])

In [14]:
def recommendation(prompt, k=5):
    target_vector = tfidf.transform([prompt])
    reduced_target_vector = svd.transform(target_vector)

    similarities = cosine_similarity(reduced_target_vector, reduced_data).flatten()
    topk = np.argpartition(similarities, -k)[-k:]
    
    return [(df.iloc[i]["name"]) for i in topk]

In [15]:
dish_prompts = [
    "I'm looking for a dairy-free breakfast dish with a hint of spice; something like a Mexican-style omelet but without cheese.",
    "Can you suggest a gluten-free Italian lunch option, preferably with a lot of fresh vegetables and no meat?",
    "I need a quick Japanese-inspired snack, but I'm allergic to shellfish—something light and vegetarian would be ideal.",
    "I'm craving a hearty vegan dinner that's Mediterranean-inspired, featuring chickpeas or lentils and lots of fresh herbs.",
    "Could you recommend a kid-friendly dessert that's nut-free but still has a touch of chocolate? Maybe something from French cuisine?",
    "I want a light Thai lunch dish, but it has to be free of soy and seafood, with a focus on fresh and crisp ingredients.",
    "I'm in the mood for a comforting American breakfast but need it to be egg-free—something like pancakes or waffles without eggs would be great.",
    "Give me an Indian-inspired dinner option that's mild in spice and free of dairy. I'd love something with a good mix of veggies.",
    "I need a quick snack that's Mediterranean-style and free of gluten and dairy, ideally something I can eat on the go.",
    "I'm looking for a festive dessert for dinner, inspired by Mexican cuisine but without any cinnamon or nuts."
]

In [16]:
def generate_dish_recs():
    result = []
    for prompt in dish_prompts:
        rec = recommendation(prompt)
        for i, dish in enumerate(rec, start=1):
            result.append(dish)
    return result

dish_recs = generate_dish_recs()
dish_recs

['Taco Bake',
 'Gluten- and Dairy-Free Chicken Enchiladas',
 'The Perfect Non-Dairy, No-Sugar Spiced Chai',
 'Best Tasting Base for Enchilidas. 6 Ingredients!',
 'Quinoa Rice Breakfast (Gluten Free, Dairy Free)',
 'Paleo Coconut Bread',
 'Asparagus  Omelette Wraps',
 'Fun Mini Muffins -Almond Meal- Flourless',
 'Italian Herb Salad Dressing',
 'Almond Snowballs',
 'Carrot and Sesame Furikake',
 'Light and Crisp Veggie Sandwich',
 'Miso-Garlic Broiled Eggplant',
 'Garnished Rice With Green Tea (Ocha-Zuke)',
 'Japanese Spring Vegetable Ragout',
 'Beef and Vegetable',
 'Roast Turkey With White Wine and Rosemary',
 'Moroccan Lentil and Chickpea Soup',
 'Darn Good 4 You Granola Bars-(With a Secret Ingredient!!)',
 'Tunisian Tomato Soup With Chickpeas and Lentils',
 'Dill Dip',
 'Double Chocolate Cranberry Cookies',
 'Quinoa Rice Breakfast (Gluten Free, Dairy Free)',
 'Cinnamon Pudding Dessert',
 'Mediterranean Chickpea Salad',
 'Thai Chilli Swordfish',
 'Green Curry Paste - Namphrik Gaeng Ke

In [17]:
def preprocess_data(df, tokenizer, max_length=128):
    # Tokenize names
    inputs = tokenizer(list(df["name"]), max_length=max_length, padding=True, truncation=True, return_tensors="pt")
    
    # Generate multi-label binarized labels
    mlb = MultiLabelBinarizer()
    df["ingredients_list"] = df["ingredients_str"].apply(lambda x: x.split())
    labels = mlb.fit_transform(df["ingredients_list"])

    return inputs, labels, mlb.classes_

bert_tokenizer = BertTokenizer.from_pretrained("distilbert-base-uncased", clean_up_tokenization_spaces=True)
inputs, labels, classes = preprocess_data(df, bert_tokenizer, max_length=128)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [18]:
class BERTIngredientClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BERTIngredientClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output
        logits = self.classifier(cls_output)
        return self.sigmoid(logits) 

In [24]:
X_train, X_test, y_train, y_test = train_test_split(inputs['input_ids'], labels, test_size=0.1, random_state=42)
attention_train, attention_val = train_test_split(inputs['attention_mask'], test_size=0.1, random_state=42)

train_dataset = TensorDataset(X_train, attention_train, torch.tensor(y_train, dtype=torch.float32))
test_dataset = TensorDataset(X_test, attention_val, torch.tensor(y_test, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


In [25]:
train_loader.__len__(), test_loader.__len__()

(139, 16)

In [26]:
num_labels = labels.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Devie: {device}")

Devie: cpu


In [27]:
bert_model = BERTIngredientClassifier(num_labels).to(device)
optimizer = optim.AdamW(bert_model.parameters(), lr=5e-5)
criterion = nn.BCELoss()

In [28]:
def train(model, train_loader, optimizer, criterion, epochs):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for input_ids, attention_mask, y_true in train_loader:
            input_ids, attention_mask, y_true = input_ids.to(device), attention_mask.to(device), y_true.to(device)
            
            optimizer.zero_grad()
            y_pred = model(input_ids, attention_mask)
            loss = criterion(y_pred, y_true)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for input_ids, attention_mask, y_true in test_loader:
            input_ids, attention_mask, y_true = input_ids.to(device), attention_mask.to(device), y_true.to(device)
            y_pred = model(input_ids, attention_mask)
            loss = criterion(y_pred, y_true)
            total_loss += loss.item()
    
    print(f"Test Loss: {total_loss / len(test_loader)}")

In [30]:
train(bert_model, train_loader, optimizer, criterion, epochs=5)
evaluate(bert_model, test_loader, criterion)

Epoch 1, Loss: 0.07946935588185736
Epoch 2, Loss: 0.04841013823588975
Epoch 3, Loss: 0.04275617432251251
Epoch 4, Loss: 0.040739836774284034
Epoch 5, Loss: 0.03981358271172578
Test Loss: 0.038481244118884206


In [36]:
with open('models/ingredient_model.pkl', 'wb') as f:
    pickle.dump(bert_model, f)

In [32]:
mlb = MultiLabelBinarizer(classes=classes)
labels = mlb.fit_transform(df["ingredients_list"])

In [33]:
def generate_ingredient_rec(model, tokenizer, dish): 
    model.eval()
    tokenized_dish = tokenizer(dish, max_length=128, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad(): 
        pred = model(tokenized_dish['input_ids'].to(device), tokenized_dish['attention_mask'].to(device))
        pred_ingedients = mlb.inverse_transform(pred.sigmoid() > 0.51)
    return pred_ingedients[0]


ingredients = generate_ingredient_rec(bert_model, bert_tokenizer, "Spicy Thai Green Curry")

In [34]:
def print_ingredients(dish, ingredients):
    print(f"Ingredients for {dish}:")
    for ingredient in ingredients:
        print(f"- {ingredient}")

print_ingredients("Spicy Thai Green Curry", generate_ingredient_rec(bert_model, bert_tokenizer, "Spicy Thai Green Curry"))

Ingredients for Spicy Thai Green Curry:
- all-purpose
- and
- bacon
- baking
- basil
- beans
- beef
- bell
- black
- boneless
- bread
- broth
- brown
- butter
- cayenne
- celery
- cheddar
- cheese
- chicken
- chili
- chocolate
- cinnamon
- clove
- cloves
- corn
- cream
- cumin
- dried
- dry
- egg
- eggs
- extract
- flakes
- flour
- fresh
- frozen
- garlic
- ginger
- green
- ground
- italian
- juice
- leaves
- lemon
- light
- milk
- mix
- mushrooms
- mustard
- nutmeg
- of
- oil
- olive
- onion
- onions
- orange
- oregano
- parmesan
- parsley
- pepper
- potatoes
- powder
- red
- rice
- salt
- sauce
- seasoning
- soda
- soup
- sour
- soy
- sugar
- thyme
- tomato
- tomatoes
- vanilla
- vegetable
- vinegar
- water
- white
- whole
- wine
- yellow


In [35]:
for dish in dish_recs:
    print_ingredients(dish, generate_ingredient_rec(bert_model, bert_tokenizer, dish))

Ingredients for Taco Bake:
- all-purpose
- and
- bacon
- baking
- basil
- beans
- beef
- bell
- black
- boneless
- bread
- broth
- brown
- butter
- cayenne
- celery
- cheddar
- cheese
- chicken
- chili
- chocolate
- cinnamon
- clove
- cloves
- corn
- cream
- cumin
- dried
- dry
- egg
- eggs
- extract
- flakes
- flour
- fresh
- frozen
- garlic
- ginger
- green
- ground
- italian
- juice
- leaves
- lemon
- light
- milk
- mix
- mushrooms
- mustard
- nutmeg
- of
- oil
- olive
- onion
- onions
- orange
- oregano
- parmesan
- parsley
- pepper
- potatoes
- powder
- red
- rice
- salt
- sauce
- seasoning
- soda
- soup
- sour
- soy
- sugar
- thyme
- tomato
- tomatoes
- vanilla
- vegetable
- vinegar
- water
- white
- whole
- wine
- yellow
Ingredients for Gluten- and Dairy-Free Chicken Enchiladas:
- all-purpose
- and
- bacon
- baking
- basil
- beans
- beef
- bell
- black
- boneless
- bread
- broth
- brown
- butter
- cayenne
- celery
- cheddar
- cheese
- chicken
- chili
- chocolate
- cinnamon
- clo